# Random Forest

기계 학습에서의 랜덤 포레스트(영어: random forest)는 분류, 회귀 분석 등에 사용되는 앙상블 학습 방법의 일종으로, 훈련 과정에서 구성한 다수의 결정 트리로부터 부류(분류) 또는 평균 예측치(회귀 분석)를 출력함으로써 동작한다. - wiki



Random forest는 ensemble(앙상블) machine learning 모델입니다. 여러개의 decision tree를 형성하고 새로운 데이터 포인트를 각 트리에 동시에 통과시키며, 각 트리가 분류한 결과에서 투표를 실시하여 가장 많이 득표한 결과를 최종 분류 결과로 선택합니다. 랜덤 포레스트가 생성한 일부 트리는 overfitting될 수 있지만, 많은 수의 트리를 생성함으로써 overfitting이 예측하는데 있어 큰 영향을 미치지 못 하도록 예방합니다.


Random Forest는 먼저 bagging이라는 과정을 거칩니다. bagging은 트리를 만들 때 training set에서 데이터를 임의로 선택하여 트리를 만듭니다. 선택된 임의의 데이터는 다음 트리를 만들 때 다시 training set으로 들어가서 다시 선택받을 수 있습니다.

In [ ]:
!pip install pycaret
!pip install jinja
!pip install markupsafe==2.0.1

In [ ]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
train_err = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_problem_data.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 머신러닝_모델링/data/system_quality_project/train_quality_data.csv')

In [ ]:
id_error = train_err[['user_id','errtype']].values
id_error

array([[10000,    15],
       [10000,    12],
       [10000,    11],
       ...,
       [24999,     4],
       [24999,     4],
       [24999,    15]])

## train data 생성

In [ ]:
error = np.zeros((15000,42))
for person_idx, err in tqdm(id_error):
  error[person_idx - 10000, err - 1] += 1

error

100%|██████████| 16554663/16554663 [00:43<00:00, 382202.15it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [ ]:
train = pd.DataFrame(data=error)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


## target data 생성

In [ ]:
problem = np.zeros(15000)
train_problem.user_id.unique()
problem[train_problem.user_id.unique()-10000] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [ ]:
target = pd.DataFrame(problem)
target

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
14995,0.0
14996,0.0
14997,1.0
14998,1.0


## Training

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=42)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((12000, 42), (3000, 42), (12000, 1), (3000, 1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()

clf.fit(X_train, y_train)

pred = clf.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

display(confusion_matrix(y_val, pred))  #confusion matrix 표시
print("Accuracy for training : %.4f" % accuracy_score(y_val, pred)) # accuracy(정확도)
print("Precision for training : %.4f" % precision_score(y_val, pred)) #precision(정밀도)
print("Recall for training : %.4f" % recall_score(y_val, pred))     #sensitivity(민감도), recall(재현율)
print("F1 score for training : %.4f" % f1_score(y_val, pred))         # F1 score(정밀도, 민감도 조화평균)
print(classification_report(y_val, pred))

array([[1534,   99],
       [ 773,  594]])

Accuracy for training : 0.7093
Precision for training : 0.8571
Recall for training : 0.4345
F1 score for training : 0.5767
              precision    recall  f1-score   support

         0.0       0.66      0.94      0.78      1633
         1.0       0.86      0.43      0.58      1367

    accuracy                           0.71      3000
   macro avg       0.76      0.69      0.68      3000
weighted avg       0.75      0.71      0.69      3000



## Hyperparameter tuning

Random Forest Hyperparameter
- n_estimator
  - 결정 트리의 개수. 많을수록 좋은 성능이 나올 수도 있지만, 무조건적이지는 않다. default = 10
- max_features
  - 데이터의 feature를 참조할 비율, 개수를 뜻한다. default = auto
- max_depth
  - 트리의 깊이를 뜻한다.
- min_samples_leaf
  - 리프노드가 되기 위한 최소한의 샘플 데이터 수
- min_samples_split
  - 노드를 분할하기 위한 최소한의 데이터 수

In [ ]:
# GridSearchCV를 사용하여 최적의 파라미터 값을 찾는다.
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators' : [100],
    'max_depth' : [6,8,10,12],
    'min_samples_leaf' : [8,12,18],
    'min_samples_split' : [8,16,20]
}

In [ ]:
clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(clf, param_grid=params, scoring="accuracy", cv=2, n_jobs=-1)
grid_cv.fit(X_train,y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]},
             scoring='accuracy')

In [ ]:
print("최고 평균 정확도 : %.4f" % grid_cv.best_score_)
print("최고 파라미터 : ", grid_cv.best_params_)

최고 평균 정확도 : 0.7877
최고 파라미터 :  {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 100}


In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=8,
                           min_samples_split=20, random_state=0)

clf.fit(X_train, y_train)

pred = clf.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

display(confusion_matrix(y_val, pred))  #confusion matrix 표시
print("Accuracy for training : %.4f" % accuracy_score(y_val, pred)) # accuracy(정확도)
print("Precision for training : %.4f" % precision_score(y_val, pred)) #precision(정밀도)
print("Recall for training : %.4f" % recall_score(y_val, pred))     #sensitivity(민감도), recall(재현율)
print("F1 score for training : %.4f" % f1_score(y_val, pred))         # F1 score(정밀도, 민감도 조화평균)
print(classification_report(y_val, pred))

array([[1563,   70],
       [ 825,  542]])

Accuracy for training : 0.7017
Precision for training : 0.8856
Recall for training : 0.3965
F1 score for training : 0.5478
              precision    recall  f1-score   support

         0.0       0.65      0.96      0.78      1633
         1.0       0.89      0.40      0.55      1367

    accuracy                           0.70      3000
   macro avg       0.77      0.68      0.66      3000
weighted avg       0.76      0.70      0.67      3000

